# The Moral Pulse of the Machine - companion notebook

Chris von Csefalvay
[](https://orcid.org/0000-0003-3131-0864)  
2023-10-27

In [1]:
import os
import openai
import urllib3
import os
import json
import jsonschema
import networkx as nx
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
import seaborn as sns
import pandas as pd
from IPython.display import display, Markdown
from netgraph import Graph

%matplotlib inline

urllib3.disable_warnings(urllib3.exceptions.NotOpenSSLWarning)

/Users/kristof.csefalvay/Developer/personal-website/venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(

In [2]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
story_prompt = (
    f"Please write me a story. The story must involve three animals of your choice, but no more of those two animals may be mammals. Make sure each animal is named and has a proper name. The story must be at least %wordlen words long and focus on the pursuit of an object (of your choice) by the three animals. Each of the animals shall exhibit a virtue, e.g. kindness or courage. Determine which virtue will be most crucial in attaining the goal.")

In [4]:
languages = ["English", "Swahili", "German", "Dutch"]

stories = []

for i in tqdm(languages):
    completion = openai.api_resources.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", 
             "content": f"You're an AI whose job it is to generate bedtime stories for children in specific languages. Currently, you are generating stories in the {i} language. The story you generate should be in {i}."},
            {"role": "user",
             "content": story_prompt.format(wordlen=70) + " Generate the first 30 words only."}
            ],
        temperature=0.9)

    stories.append(completion.choices[0].message.content)

for story in stories:
    display(Markdown("> " + (" ".join(story.split(" ")[:30]) + "...")))

In [5]:
story_prompt = (
    f"Please write me a story in Swahili. The story must involve three animals of your choice, but no more of those two animals may be mammals. Make sure each animal is named and has a proper name. The story must be at least %wordlen words long and focus on the pursuit of an object (of your choice) by the three animals. Each of the animals shall exhibit a virtue, e.g. kindness or courage. Determine which virtue will be most crucial in attaining the goal.")

completion = openai.api_resources.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": story_prompt.format(wordlen=100)}
        ])

display(Markdown(">" + "\n>\n>".join(completion.choices[0].message.content.split("\n\n")[:3])))

In [6]:
story_prompt = (
    f"Please write me a story in English. The story must involve three animals of your choice, but no more of those two animals may be mammals. Make sure each animal is named and has a proper name. The story must be at least %wordlen words long and focus on the pursuit of an object (of your choice) by the three animals. Each of the animals shall exhibit a virtue, e.g. kindness or courage. Determine which virtue will be most crucial in attaining the goal.")

completion = openai.api_resources.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": story_prompt.format(wordlen=100)}
        ])

english_story = completion.choices[0].message.content

display(Markdown(">" + "\n>\n>".join(english_story.split("\n\n")[:3])))


In [7]:
story_schema = """
{
    "$schema": "http://json-schema.org/draft-07/schema#",
    "title": "Story",
    "type": "object",
    "properties": {
        "goal": { "type": "string" },
        "protagonists": {
            "type": "array",
            "items": {
                "type": "object",
                "properties": {
                    "name": { "type": "string" },
                    "species": { "type": "string" },
                    "virtue": { "type": "string" }
                },
                "required": ["species", "virtue"]
            }
        },
        "winner": { "type": "string" }
    },
    "required": ["protagonists", "winner"]
}
"""

In [8]:
json_completion = openai.api_resources.ChatCompletion.create(
    model = "gpt-3.5-turbo",
    messages = [
        {"role": "user", "content": """I will present you with a story. You must identify the following parameters in this story:
        
        * the goal,
        * each of the protagonists, for whom you must each identify their name, their species and the virtue they embody,
        * the name of the winner - that is, the protagonist whose virtue prevailed or proved most important.
        
        Express your answer in a JSON document that validates against the following schema:""" + "\n\n" + "```json\n" + story_schema + "\n```\n\n" + "The 'winner' field should contain the name of the animal whose virtue was most crucial in attaining the goal. Only return the JSON document. Make sure the output is valid JSON." + "\n\n" + "Here is the story: \n\n" + english_story}
        ])

result = json_completion.choices[0].message.content

print(result)

{
  "goal": "Unravel the mystery of the enchanted gem",
  "protagonists": [
    {
      "name": "Oliver",
      "species": "owl",
      "virtue": "kindness"
    },
    {
      "name": "Jasper",
      "species": "snake",
      "virtue": "bravery"
    },
    {
      "name": "Felix",
      "species": "turtle",
      "virtue": "loyalty"
    }
  ],
  "winner": "Felix"
}

In [9]:
jsonschema.validate(json.loads(json_completion.choices[0].message.content), 
                    json.loads(story_schema))


In [11]:
G = nx.DiGraph()

num_of_stories: int = 10
stories_generated: int = 0
fail_counter: int = 0

with tqdm(total=num_of_stories) as pbar:
    while stories_generated < num_of_stories:
        json_completion = openai.api_resources.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = [
                {"role": "user", "content": story_prompt.format(wordlen=100) + 
                 """Express your answer in a JSON document that validates against the following schema:""" + "\n\n" + "```json\n" + story_schema + "\n```\n\n" + "The 'winner' field should contain the name of the animal whose virtue was most crucial in attaining the goal."}
                ])
        
        try:
            story = json.loads(json_completion.choices[0].message.content)
        
            # Validate JSON output against schema
            try:
                jsonschema.validate(story, json.loads(story_schema))
        
                winner = story["winner"]
                winner_virtue = [i["virtue"].lower() for i in story["protagonists"] if i["name"] == winner][0]
                
                losers = [i["virtue"].lower() for i in story["protagonists"] if i["name"] != winner]
                losing_virtues = [i for i in losers if i != winner_virtue]
                
                for node in winner_virtue, *losing_virtues:
                    if not G.has_node(node):
                        G.add_node(node, weight=0)
                        
                for i in losers:
                    if G.has_edge(winner_virtue, i):
                        G[winner_virtue][i]["weight"] += 1
                    else:
                        G.add_edge(winner_virtue, i, weight=1)
                        
                stories_generated += 1
                pbar.update(1)
                    
            except jsonschema.exceptions.ValidationError as err:
                fail_counter += 1
                continue
    
        except json.JSONDecodeError or jsonschema.exceptions.ValidationError as err:
            fail_counter += 1
            continue
        
print(f"Generated {stories_generated} stories (with {fail_counter} fails and retries).")

Generated 10 stories (with 1 fails and retries).

In [13]:
df = pd.DataFrame(index = G.nodes(), columns = G.nodes(), dtype=int).fillna(0)

for node in G:
    for neighbour, attr in G[node].items():
        df.loc[node, neighbour] = attr['weight']

In [14]:
df

In [18]:
abilities = ["Strength", "Dexterity", "Constitution", "Intelligence", "Wisdom", "Charisma"]

virtues = list(df.index)
virtues_to_abilities = {}

with tqdm(total=len(virtues)) as pbar:
    while virtues:
        current_virtue = virtues[-1]
    
        completion = openai.api_resources.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system",
                 "content": f"You are Abiliser, a system designed to determine the governing ability of particular virtues. You are given a virtue and must determine which ability governs it. The abilities are: {', '.join(abilities)}. You must choose from this list. You cannot choose any word that is not in this list. You must respond in the form of a single word denoting the governing ability. For example, if the virtue is courage, you must respond with 'Charisma' If the virtue is 'cleverness', you must respond with 'Intelligence'."},
                {"role": "user", "content": f"Virtue: {current_virtue}"}
                ])
    
        if completion.choices[0].message.content.title() in abilities:
            virtues_to_abilities[current_virtue] = completion.choices[0].message.content.title()
            virtues.pop()
            pbar.update(1)

{'strength': 'Constitution',
 'cunning': 'Intelligence',
 'determination': 'Constitution',
 'resourcefulness': 'Intelligence',
 'sharp vision': 'Dexterity',
 'loyalty': 'Wisdom',
 'cleverness': 'Intelligence',
 'curiosity': 'Intelligence',
 'patience': 'Wisdom',
 'courage': 'Charisma',
 'alertness': 'Wisdom',
 'perseverance': 'Constitution',
 'speed': 'Dexterity',
 'bravery': 'Charisma',
 'wisdom': 'Intelligence'}

In [20]:
df = pd.DataFrame(index=G.nodes(), columns=G.nodes(), dtype=int).fillna(0)

for node in G:
    for neighbour, attr in G[node].items():
        df.loc[node, neighbour] = attr['weight']

df = df.rename(columns=virtues_to_abilities, index=virtues_to_abilities)

df = df.groupby(df.columns, axis=1).sum()
df = df.groupby(df.index).sum()

df

/var/folders/7l/9md9kdsj087d62kr6f358bfh0000gp/T/ipykernel_78297/4077426324.py:9: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()

In [22]:
G = nx.DiGraph()

num_of_stories: int = 100
stories_generated: int = 0
fail_counter: int = 0

with tqdm(total=num_of_stories) as pbar:
    while stories_generated < num_of_stories:
        json_completion = openai.api_resources.ChatCompletion.create(
            model = "gpt-3.5-turbo",
            messages = [
                {"role": "user", "content": story_prompt.format(wordlen=100) + 
                 """Express your answer in a JSON document that validates against the following schema:""" + "\n\n" + "```json\n" + story_schema + "\n```\n\n" + "The 'winner' field should contain the name of the animal whose virtue was most crucial in attaining the goal."}
                ])
        
        try:
            story = json.loads(json_completion.choices[0].message.content)
        
            # Validate JSON output against schema
            try:
                jsonschema.validate(story, json.loads(story_schema))
        
                winner = story["winner"]
                winner_virtue = [i["virtue"].lower() for i in story["protagonists"] if i["name"] == winner][0]
                
                losers = [i["virtue"].lower() for i in story["protagonists"] if i["name"] != winner]
                losing_virtues = [i for i in losers if i != winner_virtue]
                
                for node in winner_virtue, *losing_virtues:
                    if not G.has_node(node):
                        G.add_node(node, weight=0)
                        
                for i in losers:
                    if G.has_edge(winner_virtue, i):
                        G[winner_virtue][i]["weight"] += 1
                    else:
                        G.add_edge(winner_virtue, i, weight=1)
                        
                stories_generated += 1
                pbar.update(1)
                    
            except jsonschema.exceptions.ValidationError as err:
                fail_counter += 1
                continue
    
        except json.JSONDecodeError or jsonschema.exceptions.ValidationError as err:
            fail_counter += 1
            continue
        
print(f"Generated {stories_generated} stories (with {fail_counter} fails and retries).")

Generated 100 stories (with 8 fails and retries).

In [27]:
df = pd.DataFrame(index=G.nodes(), columns=G.nodes(), dtype=int).fillna(0)

for node in G:
    for neighbour, attr in G[node].items():
        df.loc[node, neighbour] = attr['weight']

abilities = ["Strength", "Dexterity", "Constitution", "Intelligence", "Wisdom", "Charisma"]

virtues = list(df.index)
virtues_to_abilities = {}

with tqdm(total=len(virtues)) as pbar:
    while virtues:
        current_virtue = virtues[-1]
    
        completion = openai.api_resources.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system",
                 "content": f"You are Abiliser, a system designed to determine the governing ability of particular virtues. You are given a virtue and must determine which ability governs it. The abilities are: {', '.join(abilities)}. You must choose from this list. You cannot choose any word that is not in this list. You must respond in the form of a single word denoting the governing ability. For example, if the virtue is courage, you must respond with 'Charisma' If the virtue is 'cleverness', you must respond with 'Intelligence'."},
                {"role": "user", "content": f"Virtue: {current_virtue}"}
                ])
    
        if completion.choices[0].message.content.title() in abilities:
            virtues_to_abilities[current_virtue] = completion.choices[0].message.content.title()
            virtues.pop()
            pbar.update(1)

/var/folders/7l/9md9kdsj087d62kr6f358bfh0000gp/T/ipykernel_78297/2369364840.py:15: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  df = df.groupby(df.columns, axis=1).sum()

In [31]:
completion = openai.api_resources.ChatCompletion.create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "Can you tell me whether intelligence or charisma is more important to achieving great, noble goals?"}
        ])

display(Markdown(">" + "\n>\n>".join(completion.choices[0].message.content.split("\n\n"))))